# Стандартное обучение RecRNN модели

Этот ноутбук запускает стандартное обучение маленькой версии RecRNN модели.

## 1. Клонирование репозитория

Сначала клонируем репозиторий с кодом и данными:

In [ ]:
!git clone https://your-repo-url.git /kaggle/working/recurrent-pretraining
%cd /kaggle/working/recurrent-pretraining

## 2. Установка зависимостей

Установим необходимые библиотеки:

In [ ]:
!pip install lightning torch torchdata numpy tqdm

## 3. Настройка директорий данных

Настроим пути к данным и выходным директориям для Kaggle:

In [ ]:
import os
import shutil
from pathlib import Path

# Проверяем, есть ли данные во входном каталоге
input_dir = Path("/kaggle/input")
data_sources = list(input_dir.glob("*/data"))

if not data_sources:
    print("Данные не найдены в /kaggle/input - будем использовать собственные данные из репозитория")
    data_dir = Path("/kaggle/working/recurrent-pretraining/data")
else:
    data_dir = data_sources[0]
    print(f"Будем использовать данные из {data_dir}")

# Создаем символическую ссылку для удобства доступа
working_data_dir = Path("/kaggle/working/data")
if not working_data_dir.exists():
    os.symlink(data_dir, working_data_dir)

# Создаем директории для выходных данных
output_dir = Path("/kaggle/working/output")
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Директория данных: {data_dir}")
print(f"Рабочая ссылка на данные: {working_data_dir}")
print(f"Директория для выходных файлов: {output_dir}")

## 4. Анализ датасета

Запустим анализ данных, чтобы понять их структуру:

In [ ]:
!python analyze_dataset.py

## 5. Подготовка данных

Подготовим данные, токенизируя их для модели:

In [ ]:
!python prepare_optimized_data.py

## 6. Обучение модели с Kaggle-специфичным скриптом

Запустим обучение используя специальную версию скрипта для Kaggle:

In [ ]:
!python kaggle_run.py --hidden-size 768 --batch-size 16

## 6. Проверка результатов

Проверим файлы, созданные в процессе обучения:

In [ ]:
import os
print("Files in output directory:")
!ls -la /kaggle/working/output/

## 8. Простая проверка на тестовых примерах

Протестируем обученную модель на нескольких примерах:

In [ ]:
import torch
import json
from simple_tokenizer import SimpleTokenizer

# Загрузим несколько примеров из тестового набора
with open("/kaggle/working/data/test.json", "r") as f:
    test_examples = json.load(f)[:5]  # Берем первые 5 примеров

# Выведем примеры
for i, example in enumerate(test_examples):
    print(f"Пример {i+1}:")
    print(f"  Input: {example['input_text']}")
    print(f"  Target: {example['target_text']}")
    print(f"  Тип: {example.get('type', 'unknown')}")
    print()

## 9. Сохранение модели для дальнейшего использования

Скопируем модель в постоянное хранилище Kaggle:

In [ ]:
!mkdir -p /kaggle/working/saved_models
!cp -r /kaggle/working/output/* /kaggle/working/saved_models/